In [1]:
!pip install transformers datasets accelerate protobuf sentencepiece


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [2]:
from huggingface_hub import login 
login()

In [3]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

# You can change this to llama
model_id = "mistralai/Mistral-7B-Instruct-v0.2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16
).eval()

print(f"Loaded {model_id}")


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/596 [00:00<?, ?B/s]

...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.94G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

Loaded mistralai/Mistral-7B-Instruct-v0.2


In [4]:
from datasets import load_dataset

gsm8k = load_dataset("gsm8k", "main", split="test[:5]")
strategyqa = load_dataset("tasksource/strategy-qa", split="train[:5]")
arc = load_dataset("ai2_arc", "ARC-Challenge", split="test[:5]")

print("Loaded GSM8K, StrategyQA, ARC-Challenge")


Loaded GSM8K, StrategyQA, ARC-Challenge


In [5]:
def generate_answer(prompt, max_new_tokens=128):
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    with torch.no_grad():
        outputs = model.generate(**inputs, max_new_tokens=max_new_tokens)
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [6]:
import re

def extract_number(text):
    matches = re.findall(r"\b\d+\b", text)
    return matches[-1] if matches else "?"

gsm8k_outputs = []
for ex in gsm8k:
    prompt = f"<|user|>\n{ex['question']}\n<|assistant|>\n"
    full_output = generate_answer(prompt)
    model_answer = extract_number(full_output)
    
    gsm8k_outputs.append({
        "question": ex["question"],
        "expected_answer": ex["answer"].strip().split("####")[-1].strip(),  # gold number
        "model_reasoning": full_output,
        "model_answer": model_answer
    })

print("GSM8K Outputs:")
gsm8k_outputs


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


GSM8K Outputs:


[{'question': "Janet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?",
  'expected_answer': '18',
  'model_reasoning': "<|user|>\nJanet’s ducks lay 16 eggs per day. She eats three for breakfast every morning and bakes muffins for her friends every day with four. She sells the remainder at the farmers' market daily for $2 per fresh duck egg. How much in dollars does she make every day at the farmers' market?\n<|assistant|>\nFirst, let's find out how many eggs Janet has left after eating for breakfast and baking muffins. She eats 3 eggs for breakfast and uses 4 eggs for baking muffins. So, she uses 7 eggs daily.\n\nJanet's ducks lay 16 eggs per day. So, she has 16 - 7 = <<16-7=9>>9 eggs left to sell each day.\n\nShe sells each egg for $2. So, her daily earnings at 

In [7]:
gsm8k_correct = sum(x["model_answer"] == x["expected_answer"] for x in gsm8k_outputs)
gsm8k_acc = gsm8k_correct / len(gsm8k_outputs)
print(f"GSM8K Accuracy: {gsm8k_acc:.2%} ({gsm8k_correct}/{len(gsm8k_outputs)})")

GSM8K Accuracy: 20.00% (1/5)


In [8]:
def extract_yes_no(text):
    text = text.lower()
    if "yes" in text:
        return "yes"
    elif "no" in text:
        return "no"
    return "?"

strategyqa_outputs = []
for ex in strategyqa:
    q = ex["question"]
    correct = "yes" if ex["answer"] else "no"
    prompt = f"<|user|>\n{q}\nAnswer Yes or No.\n<|assistant|>\n"
    full_output = generate_answer(prompt)
    model_answer = extract_yes_no(full_output)

    strategyqa_outputs.append({
        "question": q,
        "expected_answer": correct,
        "model_reasoning": full_output,
        "model_answer": model_answer
    })

print("StrategyQA outputs:")
strategyqa_outputs


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


StrategyQA outputs:


[{'question': 'Are more people today related to Genghis Khan than Julius Caesar?',
  'expected_answer': 'yes',
  'model_reasoning': "<|user|>\nAre more people today related to Genghis Khan than Julius Caesar?\nAnswer Yes or No.\n<|assistant|>\nNo, it is not accurate to say that more people today are related to Genghis Khan than Julius Caesar. While it is estimated that 1 in 200 men in Central Asia are descendants of Genghis Khan, the number of descendants of Julius Caesar is much larger due to the wider geographical reach and longer time span of the Roman Empire. Additionally, the methodology used to estimate Genghis Khan's descendants is a subject of ongoing debate among scientists.",
  'model_answer': 'yes'},
 {'question': 'Could the members of The Police perform lawful arrests?',
  'expected_answer': 'no',
  'model_reasoning': '<|user|>\nCould the members of The Police perform lawful arrests?\nAnswer Yes or No.\n<|assistant|>\nNo, the members of The Police are musicians, not law enf

In [9]:
strategyqa_correct = sum(x["model_answer"] == x["expected_answer"] for x in strategyqa_outputs)
strategyqa_acc = strategyqa_correct / len(strategyqa_outputs)
print(f"StrategyQA Accuracy: {strategyqa_acc:.2%} ({strategyqa_correct}/{len(strategyqa_outputs)})")

StrategyQA Accuracy: 40.00% (2/5)


In [10]:
def extract_letter(text):
    text = text.strip().upper()
    for ch in ["A", "B", "C", "D", "E"]:
        if ch in text:
            return ch
    return "?"

arc_outputs = []
for ex in arc:
    q = ex['question']
    choices_text = ex['choices']['text']
    choices_label = ex['choices']['label']
    correct = ex["answerKey"]
    
    # Format: label: choice text
    formatted_choices = "\n".join([f"{label}. {text}" for label, text in zip(choices_label, choices_text)])
    
    prompt = f"<|user|>\n{q}\n{formatted_choices}\nAnswer with A, B, C, D, or E.\n<|assistant|>\n"
    full_output = generate_answer(prompt)
    model_answer = extract_letter(full_output)

    arc_outputs.append({
        "question": q,
        "expected_answer": correct,
        "model_reasoning": full_output,
        "model_answer": model_answer
    })

print("ARC outputs:")
arc_outputs


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


ARC outputs:


[{'question': 'An astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?',
  'expected_answer': 'C',
  'model_reasoning': '<|user|>\nAn astronomer observes that a planet rotates faster after a meteorite impact. Which is the most likely effect of this increase in rotation?\nA. Planetary density will decrease.\nB. Planetary years will become longer.\nC. Planetary days will become shorter.\nD. Planetary gravity will become stronger.\nAnswer with A, B, C, D, or E.\n<|assistant|>\nC. Planetary days will become shorter.\n\nThe impact of a meteorite could transfer energy to the planet, causing an increase in its rotational speed. This would result in shorter planetary days. The other options are not directly related to the given information.',
  'model_answer': 'A'},
 {'question': 'A group of engineers wanted to know how different building designs would respond during an earthquake. They made several models of b

In [11]:
arc_correct = sum(x["model_answer"] == x["expected_answer"] for x in arc_outputs)
arc_acc = arc_correct / len(arc_outputs)
print(f"ARC Accuracy: {arc_acc:.2%} ({arc_correct}/{len(arc_outputs)})")

ARC Accuracy: 0.00% (0/5)


In [12]:
import os, json

os.makedirs("outputs_mistral", exist_ok=True)

with open("outputs_mistral/gsm8k_outputs.json", "w") as f:
    json.dump(gsm8k_outputs, f, indent=2)

with open("outputs_mistral/strategyqa_outputs.json", "w") as f:
    json.dump(strategyqa_outputs, f, indent=2)

with open("outputs_mistral/arc_outputs.json", "w") as f:
    json.dump(arc_outputs, f, indent=2)

print("Mistral outputs saved to outputs_mistral/")


Mistral outputs saved to outputs_mistral/
